In [1]:
from pathlib import Path
import polars as pl
import os
import torch
import random
import numpy as np

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(32)

In [3]:
class CFG:
    path = Path('../data/')
    split_path = Path('../data/split/')
os.makedirs(CFG.split_path, exist_ok=True)


In [4]:
name = "v0"
df = pl.read_parquet(CFG.path/'train_data.parquet', use_pyarrow=True)
seq_ids_206 = df.filter(df['sequence'].apply(len) == 206).select("sequence_id").unique().to_series().to_list()
seq_ids_177 = df.filter(df['sequence'].apply(len) == 177).select("sequence_id").unique().to_series().to_list()
seq_ids_170 = df.filter(df['sequence'].apply(len) == 170).select("sequence_id").unique().to_series().to_list()
valid_ids_206 = seq_ids_206
valid_ids_177 = np.random.choice(seq_ids_177, int(0.03 * len(seq_ids_177)), replace=False).tolist()
valid_ids_170 = seq_ids_170
valid_seq_ids = valid_ids_206 + valid_ids_177 + valid_ids_170
valid_df = df.filter(pl.col('sequence_id').is_in(valid_seq_ids))
train_df = df.filter(~pl.col('sequence_id').is_in(valid_seq_ids))
valid_df.to_pandas().to_parquet(CFG.split_path/f'valid_data_{name}.parquet')
train_df.to_pandas().to_parquet(CFG.split_path/f'train_data_{name}.parquet')

In [5]:
len(valid_ids_206), len(valid_ids_177), len(valid_ids_170)

(2499, 23525, 15000)